In [8]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [25]:
adata = sc.read('/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_sctNormalized_WNN_embedded_initialAnno_new_annotation.h5ad')

In [26]:
adata

AnnData object with n_obs × n_vars = 16953 × 14663
    obs: 'n_counts', 'sample', 'int_id', 'reporter', 'log_cell_probs', 'cell_barcodes_du', 'cell_barcodes_cr', 'cell_barcodes', 'log_counts', 'n_genes', 'log_genes', 'mt_frac', 'rp_frac', 'ambi_frac', 'total_counts_rank', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'leiden', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'log_nCount_ATAC', 'log_nFeature_ATAC', 'scrublet_scores', 'scrublet_doublets', 'scrublet_doublets_cat', 'dd_doublets', 'dd_scores', 'dd_log_p_values', 'dd_voting_average', 'dd_-log_p_values', 'dd_doublets_cat', 'pANN', 'DF_classifications_1', 'DF_classifications_2', 'pANN_SCT', 'DF_SCT_classifications_1', 'DF_SCT_classifications_2', 'df_doublets',

In [27]:
for layer in adata.layers:
    if not isinstance(adata.layers[layer], csr_matrix):
        adata.layers[layer] = csr_matrix(adata.layers[layer])

In [28]:
del adata.obsm["umap"]
del adata.obsm["umap_ATAC"]
del adata.obsm["umap_GEX"]


In [29]:
obs_cols_to_del = ['leiden_gex_graph', 'leiden_ATAC_graph', 'leiden_wnn_graph', 'umap_GEX_1', 'umap_GEX_2', 
                   'umap_ATAC_1', 'umap_ATAC_2', 'leiden_sub', 'leiden_sub1', 'leiden_sub2', 'leiden_sub3', 
                   'leiden_sub4', 'leiden_sub5', 'leiden_sub6', 'leiden_sub7', 'leiden_sub8', 'leiden_sub9', 
                   'leiden_sub10', 'leiden_sub11', 'leiden_sub12', 'leiden_sub13', 'leiden_sub14', 'leiden_sub15', 
                   'leiden_sub16', 'leiden_sub17', 'leiden_sub18', 'int_id', 'reporter', 'log_cell_probs', 
                   'cell_barcodes_du', 'cell_barcodes_cr', 'cell_barcodes', 'log_counts',
                  'scDblFinder.score', 'hybrid_class', 'hybrid_score', 'scDblFinder.class.sct', 'scDblFinder.score.sct', 'hybrid_class_sct', 'hybrid_score_sct', 'sdf_doublets', 'scds_doublets']


In [30]:
uns_to_del = ['dendrogram_leiden_sub1', 'dendrogram_leiden_sub12', 'dendrogram_leiden_sub14', 'dendrogram_leiden_sub15', 
              'dendrogram_leiden_sub16', 'dendrogram_leiden_sub17', 'dendrogram_leiden_sub18', 'dendrogram_leiden_sub2', 
              'dendrogram_leiden_sub3', 'dendrogram_leiden_sub4', 'dendrogram_leiden_sub5', 'dendrogram_leiden_sub6', 
              'dendrogram_leiden_sub7', 'dendrogram_leiden_sub8', 'dendrogram_leiden_sub9', 'final_doublets_cat_colors', 
              'leiden_ATAC_graph_colors', 'leiden_colors', 'leiden_gex_graph_colors', 'leiden_sub10_colors', 
              'leiden_sub11_colors', 'leiden_sub12_colors', 'leiden_sub13_colors', 'leiden_sub14_colors', 
              'leiden_sub15_colors', 'leiden_sub16_colors', 'leiden_sub17_colors', 'leiden_sub18_colors', 
              'leiden_sub1_colors', 'leiden_sub2_colors', 'leiden_sub3_colors', 'leiden_sub4_colors', 'leiden_sub5_colors', 
              'leiden_sub6_colors', 'leiden_sub7_colors', 'leiden_sub8_colors', 'leiden_sub9_colors', 'leiden_sub_colors', 
              'leiden_wnn_graph_colors']

In [31]:
for col in obs_cols_to_del:
    del adata.obs[col]

In [32]:
for el in uns_to_del:
    del adata.uns[el]

In [67]:
adata.X = csr_matrix(np.round(np.exp(adata.layers["raw_counts"].A)-1))

In [69]:
del adata.layers["raw_counts"]

In [70]:
adata_lsi = sc.read("/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_lsi_peaks_to_adata.h5ad")

In [71]:
adata_lsi

AnnData object with n_obs × n_vars = 16978 × 1
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'nCount_peaks', 'nFeature_peaks', 'sample', 'peaks_snn_res.0.8', 'seurat_clusters'
    obsm: 'lsi_full', 'lsi_red'

In [72]:
def rename_index(x):
    prefix, barcode = x.split("_")
    if prefix == "E14-5":
        new_suffix="0"
    elif prefix == "E15-5":
        new_suffix="1"
    else:
        raise ValueError("no match")
    return barcode+"-"+new_suffix

In [73]:
adata_lsi.obs["old_index"] = adata_lsi.obs.index
adata_lsi.obs["index_adapted"] = adata_lsi.obs["old_index"].apply(rename_index)

In [74]:
adata_lsi_adapted = adata_lsi[adata_lsi.obs["index_adapted"].isin(adata.obs.index)]

In [75]:
adata_lsi_adapted.obs.set_index("index_adapted", inplace=True)

In [76]:
np.all(adata_lsi_adapted.obs.index == adata.obs.index)

True

In [77]:
adata.obsm["lsi_full"] = adata_lsi_adapted.obsm["lsi_full"]
adata.obsm["lsi_red"] = adata_lsi_adapted.obsm["lsi_red"]

In [78]:
adata

AnnData object with n_obs × n_vars = 16953 × 14663
    obs: 'n_counts', 'sample', 'n_genes', 'log_genes', 'mt_frac', 'rp_frac', 'ambi_frac', 'total_counts_rank', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'leiden', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'log_nCount_ATAC', 'log_nFeature_ATAC', 'scrublet_scores', 'scrublet_doublets', 'scrublet_doublets_cat', 'dd_doublets', 'dd_scores', 'dd_log_p_values', 'dd_voting_average', 'dd_-log_p_values', 'dd_doublets_cat', 'pANN', 'DF_classifications_1', 'DF_classifications_2', 'pANN_SCT', 'DF_SCT_classifications_1', 'DF_SCT_classifications_2', 'df_doublets', 'scDblFinder.class', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'batch', 'size_factors', 'mt_fra

In [79]:
adata_cr_peaks = sc.read("/Users/dominik.klein/data/E14-E15_adata_filtered_rmDoublets_cellranger_peaks.h5ad")


/Users/dominik.klein/opt/anaconda3/envs/mm_best_muon_full/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [80]:
adata_cr_peaks.obs["index_adapted"] = adata_cr_peaks.obs["Row.names"].apply(rename_index)

In [81]:
adata_cr_peaks_adapted = adata_cr_peaks[adata_cr_peaks.obs["index_adapted"].isin(adata.obs.index)]

In [82]:
adata_cr_peaks_adapted


View of AnnData object with n_obs × n_vars = 16953 × 228321
    obs: 'Row.names', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'nCount_ATAC', 'nFeature_ATAC', 'nucleosome_signal', 'nucleosome_percentile', 'TSS.enrichment', 'TSS.percentile', 'doublet_calls', 'final_doublets', 'final_doublets_cat', 'nCount_peaks', 'nFeature_peaks', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'peaks.weight', 'sample', 'celltype_new', 'index_adapted'

In [83]:
adata_cr_peaks_adapted.obs.set_index("index_adapted", inplace=True)

In [84]:
np.all(adata_lsi_adapted.obs.index == adata.obs.index)

True

In [85]:
adata.obsm["cellranger_peaks"] = adata_cr_peaks_adapted.X

In [86]:
adata.write("/Users/dominik.klein/data/pancreas_multiome_2022.h5ad")